In [1]:
import numpy as np
import pandas as pd # for loading data
import re # for removing special characters
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords # For removing stopwords

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/nicktehrany/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nicktehrany/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv("dataset.csv")
df = df.drop(['Singer', 'Date', 'Tags'], axis=1)

In [3]:
def get_index(word, word_list):
    for index, value in enumerate(word_list):
        if word == value:
            return index

## Creating a word list of all possible words from all Song Lyrics

In [4]:
corpus = []
lyrics = df['Lyrics']
sw = stopwords.words("english")
word_list = []
for i, value in df.iterrows():
    text = re.sub('[^a-zA-Z]', ' ', str(lyrics[i])) # removes special characters
    text = text.lower() # lowercases everything
    text = text.split() # splits words
    text = [wordnet_lemmatizer.lemmatize(word, pos="v") for word in text if not word in set(sw)]
    formatted_text = ""
    for word in text:
        if word not in word_list:
            word_list.append(word)
        formatted_text+=word+" "
    corpus.append(formatted_text)
    df['Lyrics'][i] = formatted_text
lyrics = corpus

#Cleaning variables to save some memory
del corpus, sw, text

### Function that creates a one-hot vector for the given text from the word_list

In [5]:
from tensorflow.keras.utils import to_categorical

def one_hot(text, word_list):
    word_vector = np.zeros(shape=(1,len(word_list)))
    text = text.split() # splits words
    for w in text:
        word_vector[0, get_index(w, word_list)] = 1
    return word_vector[0]

/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/frame


## Creating one-hot vector for genres

In [6]:
from tensorflow.keras.utils import to_categorical
genres = []

for i, value in df.iterrows():
    genre = df['Genre'][i]
    index = 0
    if genre == '[\'Pop\']': index = 0
    elif genre == '[\'Rock\']': index = 1
    elif genre == '[\'Hip-Hop/Rap\']': index = 2
    elif genre == '[\'Country\']': index = 3
    elif genre == '[\'R&B/Soul\']': index = 4
    elif genre == '[\'Metal\']': index = 5
    elif genre == '[\'Alternative/Indie\']': index = 6
    elif genre == '[\'Folk\']': index = 7
    genres.append(index)

genres = to_categorical(genres, 8)

## Replace all the lyrics with their respctive vector of word occurrences to be used as the input layer for the neural network

In [7]:
all_lyrics = np.zeros(shape=(len(df),len(word_list)))
for index, value in df.iterrows():
    all_lyrics[index] = one_hot(str(df['Lyrics'][index]), word_list)
print(all_lyrics.shape)

#Cleaning variables to save some memory
del df

(14400, 40677)


## **Don't rerun the cell below, as this will result in new test/train sets. Can rerun all other cells except this one. Probably also don't want to rerun all cells above since they always do the exact same thing**

In [8]:
from sklearn.model_selection import train_test_split

lyrics_train, lyrics_test, genre_train, genre_test = train_test_split(all_lyrics, genres, train_size = 0.85, test_size = 0.15, shuffle=True) 

#Cleaning Lists, since they take up almost 5GB
del all_lyrics, genres

# Neural Network stuff 
## **Modify and run only the cells below to change the neural network**

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(128, input_shape=(len(word_list),))) # first dense layer, 32 hidden units
model.add(Activation('relu'))            # activation layer
model.add(Dense(20)) 
model.add(Dense(8))                     # second dense layer
model.add(Activation('softmax'))         # output class probabilities

model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               5206784   
_________________________________________________________________
activation (Activation)      (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                2580      
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 168       
_________________________________________________________________
activation_1 (Activation)    (None, 8)                 0         
Total params: 5,209,532
Trainable params: 5,209,532
Non-trainable params: 0
____________________________________________________________

In [10]:
from tensorflow.keras.optimizers import SGD, Adam

optimizer = Adam(lr=0.001) # lr is the learning rate
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.fit(lyrics_train, genre_train, epochs=10, validation_split=1/10);

Train on 11016 samples, validate on 1224 samples
Epoch 1/10
11016/11016 [==============================] - 5s 493us/sample - loss: 1.4174 - acc: 0.4812 - val_loss: 1.2395 - val_acc: 0.5637
Epoch 2/10
11016/11016 [==============================] - 5s 472us/sample - loss: 0.6513 - acc: 0.7914 - val_loss: 1.3990 - val_acc: 0.5580
Epoch 3/10
11016/11016 [==============================] - 5s 473us/sample - loss: 0.3167 - acc: 0.9078 - val_loss: 1.6154 - val_acc: 0.5547
Epoch 4/10
11016/11016 [==============================] - 5s 474us/sample - loss: 0.1636 - acc: 0.9615 - val_loss: 1.9057 - val_acc: 0.5482
Epoch 5/10
11016/11016 [==============================] - 5s 475us/sample - loss: 0.0980 - acc: 0.9828 - val_loss: 2.1230 - val_acc: 0.5253
Epoch 6/10
11016/11016 [==============================] - 5s 479us/sample - loss: 0.0712 - acc: 0.9883 - val_loss: 2.1456 - val_acc: 0.5408
Epoch 7/10
11016/11016 [==============================] - 5s 479us/sample - loss: 0.0522 - acc: 0.9906 - val_lo

## Testing the Neural Network's performace

In [12]:
results = model.evaluate(lyrics_test, genre_test, batch_size=128)

2160/2160 [==============================] - 0s 120us/sample - loss: 2.6641 - acc: 0.4954
